In [1]:
def classify_var_same_sub(subid='sub', trainList=['mem','mixed','motor'], predictList=['mem','mixed','motor']):
    import pandas as pd
    import numpy as np
    from mvpa2.datasets import dataset_wizard, vstack
    from mvpa2.generators.partition import NFoldPartitioner
    from mvpa2.measures.base import CrossValidation
    from mvpa2.clfs.meta import MappedClassifier
    from mvpa2.clfs.svm import LinearCSVMC
    import reshape
    import scipy.io
    #make an empty df to store accuracy scores from each run
    accTable=[]
    for i in trainList:
        #Load task FC
        taskFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/'+i+'/'+subid+'_parcel_corrmat.mat')
        #Load rest
        restFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/rest/'+subid+'_parcel_corrmat.mat')
        #Create a training dataset targets 1/0 ==task/rest, chunk=#days
        taskdf=dataset_wizard(samples=taskFC, targets=1, chunks=range(10))
        restdf=dataset_wizard(samples=restFC, targets=0, chunks=range(10))
        training=vstack((taskdf, restdf))
        #Linear SVM with cross validation scheme leave one out, output is the accuracy
        cv=CrossValidation(LinearCSVMC(), NFoldPartitioner(), 
                      enable_ca=['stats'],
                      errorfx=lambda p, t: np.mean(p == t))
        #Train the classifier         
        cv.train(training)
        acc_scores_per_task=[]
        #Lets loop through all scenerios for sub and task of interest
        for j in predictList:
            #Load testing set
            test_taskFC=reshape.matFiles('/Users/Alexis/Box/Quest_Backup/MSC/TaskFC/FC_Parcels/'+j+'/'+subid+'_parcel_corrmat.mat')
            #Create testing dataset
            testing=dataset_wizard(samples=test_taskFC, targets=1, chunks=range(10))
            #Time to test
            pre=cv(testing)
            #prediction accuracy
            acc=np.mean(pre)
            acc_scores_per_task.append(acc)
        tmp_df=pd.DataFrame({'Test_variables':predictList, i:acc_scores_per_task}).set_index('Test_variables')
        accTable.append(tmp_df)
    accTable=pd.concat(accTable, axis=1)
    accTable.to_csv('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/MVPA/'+subid+'accTable_same_sub.csv')

In [2]:
subs=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC08','MSC09','MSC10']
for sub in subs:
    classify_var_same_sub(sub)

//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/misc/attrmap.py:153: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  if not np.issubdtype(attr.dtype, str) and not self.mapnumeric:
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:215: FutureWarning: Conversion of the second argument of issubdtype from `'c'` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype('c').type`.
  if ( np.issubdtype(self.ca.trained_targets.dtype, 'c') or
//anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/mvpa2/clfs/libsvmc/svm.py:216: FutureWarning: Conversion of the second argument of issubdtype from `'U'` to `unicode` is deprecated. In future, it will be treated as `np.unicode_ == np.dtype('U').type`.
  np.issubdtype(self.ca.trained_targets.dtype, 'U') ):
